## Load document 

In [24]:
from langchain_community.document_loaders import TextLoader

doc_path = "/Users/shakibibnashameem/Documents/Practice/LangChain/rag/odessy.txt"

loader = TextLoader(file_path=doc_path)
doc = loader.load()

## Split into chunks

In [25]:
from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter(chunk_size = 2500, chunk_overlap = 100)
chunks = splitter.split_documents(doc)

## Embeddings

In [26]:
from langchain_ollama import OllamaEmbeddings

embedding = OllamaEmbeddings(
    model='nomic-embed-text'
)

## Setup Vector DB

In [27]:
from langchain_community.vectorstores import Chroma

vector_db = Chroma.from_documents(
    documents = chunks,
    embedding=embedding
)

## Setup Prompt

In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt_template = """
        You are an helpful AI assistant. Answer the question below based on the the context, If you 
        don't know the answer, replay that I don't know the answer. 

        Context : {context}
        Question : {question}
"""
prompt = ChatPromptTemplate.from_template(prompt_template)

## Retriever

In [29]:
retriever = vector_db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 4, "score_threshold": 0.3},
    prompt = prompt
)

In [30]:
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


model = ChatOllama(model='llama3.1')

chain = (
    {'context' : retriever, 'question' : RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

res = chain.invoke("Who is Odysseus's wife?")

print(res)

Actually, in this passage, there is no mention of Odysseus's wife. The text mentions Alcinous and his wife Arete, but they are not related to Odysseus. In fact, the story appears to be a separate one, where Minerva (or Athena) visits Athens and Ulysses (Odysseus) visits the kingdom of Phaeacia.

However, in Homer's Odyssey, Odysseus is depicted as being married to Penelope.
